<a href="https://colab.research.google.com/github/MicroprocessorX069/Basic-LSTM-Types/blob/master/Basic_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print("Tensorflow version: ",tf.__version__)

In [2]:
import os
path=os.getcwd()

text=open(path+'/Iliad_v3.txt','rb').read().decode(encoding='utf-8')
print("Text is {} characters long".format(len(text)))

Text is 886809 characters long


In [3]:
words=[w for w in text.split(' ') if w.strip()!='' or w=='\n']
print("Text is {} words long".format(len(words)))

Text is 153260 words long


In [4]:
vocab=sorted(set(text))
print("There are {} unique characters".format(len(vocab)))
char2int={c:i for i,c in enumerate(vocab)}
int2char=np.array(vocab)
print("Vector:\n")
for char,_ in zip(char2int,range(len(vocab))):
  print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

There are 34 unique characters
Vector:

 '\n':   0,
 ' ' :   1,
 '!' :   2,
 "'" :   3,
 ',' :   4,
 '-' :   5,
 '.' :   6,
 '?' :   7,
 'a' :   8,
 'b' :   9,
 'c' :  10,
 'd' :  11,
 'e' :  12,
 'f' :  13,
 'g' :  14,
 'h' :  15,
 'i' :  16,
 'j' :  17,
 'k' :  18,
 'l' :  19,
 'm' :  20,
 'n' :  21,
 'o' :  22,
 'p' :  23,
 'q' :  24,
 'r' :  25,
 's' :  26,
 't' :  27,
 'u' :  28,
 'v' :  29,
 'w' :  30,
 'x' :  31,
 'y' :  32,
 'z' :  33,


Each characted mapped as a no.

In [7]:
text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

"  achilles' wrath, to greece the direful spring\n  of woes unnumber'd, heavenly goddess, sing!\n  that"
 mapped to integers:
 [ 1  1  8 10 15 16 19 19 12 26  3  1 30 25  8 27 15  4  1 27 22  1 14 25
 12 12 10 12  1 27 15 12  1 11 16 25 12 13 28 19  1 26 23 25 16 21 14  0
  1  1 22 13  1 30 22 12 26  1 28 21 21 28 20  9 12 25  3 11  4  1 15 12
  8 29 12 21 19 32  1 14 22 11 11 12 26 26  4  1 26 16 21 14  2  0  1  1
 27 15  8 27]


Creating training and validation sets

In [11]:
tr_text = text_as_int[:704000] 
val_text = text_as_int[704000:] 
print("Total size: {}, Train size : {}, Test size:{}" \
.format(text_as_int.shape, tr_text.shape, val_text.shape))

Total size: (886809,), Train size : (704000,), Test size:(182809,)


Hyperparameters

In [0]:
batch_size=64
buffer_size=10000
embedding_dim=256
epoch=50
seq_length=200
examples_per_epoch=len(text)//seq_length
rnn_units=1024
vocab_size=len(vocab)

Slicing the dataset in batchsizes, shuffling and mapping input to respective target. 

In [0]:
tr_char_dataset=tf.data.Dataset.from_tensor_slices(tr_text) #what does tensor slices do
val_char_dataset=tf.data.Dataset.from_tensor_slices(val_text) 
tr_sequences=tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences=val_char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
  input_text=chunk[:-1]
  target_text=chunk[1:]
  return input_text,target_text

tr_dataset=tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset=val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)


https://towardsdatascience.com/generating-text-with-tensorflow-2-0-6a65c7bdc568